<a href="https://www.kaggle.com/code/shireenbanoa/cafe-sales-data-cleanup?scriptVersionId=219016926" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cafe-sales-dirty-data-for-cleaning-training/dirty_cafe_sales.csv


In [2]:
df=pd.read_csv('/kaggle/input/cafe-sales-dirty-data-for-cleaning-training/dirty_cafe_sales.csv')


### Approach Intro

The approach is to first focus on numerical values such as `total_spent`, `price_per_unit`, and `quantity`, as one of these can be derived from the other two. For instance, if we have the values of `quantity` and `price_per_unit`, we can compute `total_spent` using the formula:

total_spent = quantity * price_per_unit

 Example:


In the above table, the missing `total_spent` values can be calculated as:

- For Coffee: \( 2 \* 5.00 = 10.00 \)
- For Sandwich: \( 1 * 7.50 = 7.50 \)

Additionally, since the `item` column contains a single item per transaction, it can be leveraged to determine the `price_per_unit` for each unique item if it's consistent across transactions.

Once the numerical values are handled, we will proceed to clean up other columns such as `transaction_date`, `location`, and `payment_method` to align with our specific use case requirements. This may include formatting dates, standardizing location names, and categorizing payment methods for better analysis.


In [3]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB


In [4]:
# check if any row has all the values as na
df[df.isna().all(axis=1)]


,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date


In [5]:
df.columns = [col.replace(' ', '_') for col in df.columns]
df.columns = [col.lower() for col in df.columns]
df.columns

Index(['transaction_id', 'item', 'quantity', 'price_per_unit', 'total_spent',
       'payment_method', 'location', 'transaction_date'],
      dtype='object')

In [6]:
# convert the data type of quantity, price per unit , total spent
df['quantity'].unique()


array(['2', '4', '5', '3', '1', 'ERROR', 'UNKNOWN', nan], dtype=object)

In [7]:
# Need to replace ERROR , UNKNOWN with nan
error_string = ['ERROR','UNKNOWN']
df['quantity'] = df['quantity'].apply(lambda x: np.nan if (x in error_string) else x)

In [8]:
# convert to float
df['quantity'] = df['quantity'].astype(float)

In [9]:
df.info()
df['quantity'].unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_id    10000 non-null  object 
 1   item              9667 non-null   object 
 2   quantity          9521 non-null   float64
 3   price_per_unit    9821 non-null   object 
 4   total_spent       9827 non-null   object 
 5   payment_method    7421 non-null   object 
 6   location          6735 non-null   object 
 7   transaction_date  9841 non-null   object 
dtypes: float64(1), object(7)
memory usage: 625.1+ KB


array([ 2.,  4.,  5.,  3.,  1., nan])

In [10]:
# clean price per unit
df['price_per_unit'] = df['price_per_unit'].apply(lambda x: np.nan if (x in error_string) else x)
df['price_per_unit'].unique()

array(['2.0', '3.0', '1.0', '5.0', '4.0', '1.5', nan], dtype=object)

In [11]:
df['price_per_unit'] = df['price_per_unit'].astype(float)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_id    10000 non-null  object 
 1   item              9667 non-null   object 
 2   quantity          9521 non-null   float64
 3   price_per_unit    9467 non-null   float64
 4   total_spent       9827 non-null   object 
 5   payment_method    7421 non-null   object 
 6   location          6735 non-null   object 
 7   transaction_date  9841 non-null   object 
dtypes: float64(2), object(6)
memory usage: 625.1+ KB


In [12]:
df['total_spent'] = df['total_spent'].apply(lambda x: np.nan if (x in error_string) else x)
df['total_spent'].unique()

array(['4.0', '12.0', nan, '10.0', '20.0', '9.0', '16.0', '15.0', '25.0',
       '8.0', '5.0', '3.0', '6.0', '2.0', '1.0', '7.5', '4.5', '1.5'],
      dtype=object)

In [13]:
df['total_spent'] = df['total_spent'].astype(float)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_id    10000 non-null  object 
 1   item              9667 non-null   object 
 2   quantity          9521 non-null   float64
 3   price_per_unit    9467 non-null   float64
 4   total_spent       9498 non-null   float64
 5   payment_method    7421 non-null   object 
 6   location          6735 non-null   object 
 7   transaction_date  9841 non-null   object 
dtypes: float64(3), object(5)
memory usage: 625.1+ KB


In [14]:
df['item'].unique()

array(['Coffee', 'Cake', 'Cookie', 'Salad', 'Smoothie', 'UNKNOWN',
       'Sandwich', nan, 'ERROR', 'Juice', 'Tea'], dtype=object)

In [15]:
# we can fix quantity column by using total_spent/price per unit
updated_rows = df.apply(lambda row: (row['total_spent']/row['price_per_unit']) if pd.isna(row['quantity']) else row, axis = 1)
df['quantity'] = updated_rows['quantity']

In [16]:
#filled the missing values (NA's) in the `quantity` column, which originally was 9521 to 9962.

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_id    10000 non-null  object 
 1   item              9667 non-null   object 
 2   quantity          9962 non-null   float64
 3   price_per_unit    9467 non-null   float64
 4   total_spent       9498 non-null   float64
 5   payment_method    7421 non-null   object 
 6   location          6735 non-null   object 
 7   transaction_date  9841 non-null   object 
dtypes: float64(3), object(5)
memory usage: 625.1+ KB


In [17]:
#fix price per unit column by using total_spent/quantity

df['price_per_unit']= df.apply(lambda row: (row['total_spent']/row['quantity']) if pd.isna(row['price_per_unit']) else row, axis = 1)['price_per_unit']

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_id    10000 non-null  object 
 1   item              9667 non-null   object 
 2   quantity          9962 non-null   float64
 3   price_per_unit    9962 non-null   float64
 4   total_spent       9498 non-null   float64
 5   payment_method    7421 non-null   object 
 6   location          6735 non-null   object 
 7   transaction_date  9841 non-null   object 
dtypes: float64(3), object(5)
memory usage: 625.1+ KB


In [19]:
# fix total spent column by using quantity * price_per_unit
df['total_spent'] = df.apply(lambda row: (row['price_per_unit']*row['quantity']) if pd.isna(row['total_spent']) else row, axis = 1)['total_spent']

In [20]:
df[df['total_spent'].isna()]

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,transaction_id,item,quantity,price_per_unit,total_spent,payment_method,location,transaction_date
65,TXN_4987129,Sandwich,3.0,NaN,NaN,NaN,In-store,2023-10-20
236,TXN_8562645,Salad,NaN,5.0,NaN,NaN,In-store,2023-05-18
278,TXN_3229409,Juice,NaN,3.0,NaN,Cash,Takeaway,2023-04-15
641,TXN_2962976,Juice,NaN,3.0,NaN,NaN,NaN,2023-03-17
738,TXN_8696094,Sandwich,NaN,4.0,NaN,NaN,Takeaway,2023-05-14
1674,TXN_9367492,Tea,2.0,NaN,NaN,Cash,In-store,2023-06-19
1761,TXN_3611851,NaN,4.0,NaN,NaN,Credit Card,NaN,2023-02-09
2229,TXN_8498613,Sandwich,2.0,NaN,NaN,NaN,NaN,2023-11-08
2289,TXN_7524977,UNKNOWN,4.0,NaN,NaN,ERROR,NaN,2023-12-09
2585,TXN_1259340,Tea,3.0,NaN,NaN,Digital Wallet,NaN,2023-02-24


In [21]:
df[['item','price_per_unit']].drop_duplicates().dropna() 


,item,price_per_unit
0,Coffee,2.0
1,Cake,3.0
2,Cookie,1.0
3,Salad,5.0
5,Smoothie,4.0
6,UNKNOWN,3.0
7,Sandwich,4.0
14,ERROR,1.5
17,Juice,3.0
31,UNKNOWN,1.0


In [22]:
df['item'] = df['item'].apply(lambda item: np.nan if item in error_string else item)

In [23]:
values_to_replace =df[['item','price_per_unit']].drop_duplicates().dropna()
values_to_replace

,item,price_per_unit
0,Coffee,2.0
1,Cake,3.0
2,Cookie,1.0
3,Salad,5.0
5,Smoothie,4.0
7,Sandwich,4.0
17,Juice,3.0
42,Tea,1.5


In [24]:
# We need to make use values_to_replace variable to fix these both columns that is ['Item','Price Per Unit']
item_price = values_to_replace.set_index('item').to_dict()['price_per_unit']
print(f"item_price : {item_price}")
print("\n")

# here, we have to pick either Smoothie or Sandwich whenver we encounter 4 as dictionary can not store duplicate keys
price_item= {v:k for k,v in item_price.items()}
print(f"price_item : {price_item}")
df.info()

item_price : {'Coffee': 2.0, 'Cake': 3.0, 'Cookie': 1.0, 'Salad': 5.0, 'Smoothie': 4.0, 'Sandwich': 4.0, 'Juice': 3.0, 'Tea': 1.5}


price_item : {2.0: 'Coffee', 3.0: 'Juice', 1.0: 'Cookie', 5.0: 'Salad', 4.0: 'Sandwich', 1.5: 'Tea'}
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_id    10000 non-null  object 
 1   item              9031 non-null   object 
 2   quantity          9962 non-null   float64
 3   price_per_unit    9962 non-null   float64
 4   total_spent       9960 non-null   float64
 5   payment_method    7421 non-null   object 
 6   location          6735 non-null   object 
 7   transaction_date  9841 non-null   object 
dtypes: float64(3), object(5)
memory usage: 625.1+ KB


In [25]:
# drop any records where both the `item` and `price_per_unit` are empty, as 
# these entries cannot be identified or used for further analysis (in my analysis atleast).

df.dropna(subset=['item','price_per_unit'],axis = 'index', how = 'all', inplace= True )

In [26]:
# dropped around 6 rows where item and price per unit both are null as

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9994 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_id    9994 non-null   object 
 1   item              9031 non-null   object 
 2   quantity          9959 non-null   float64
 3   price_per_unit    9962 non-null   float64
 4   total_spent       9957 non-null   float64
 5   payment_method    7417 non-null   object 
 6   location          6732 non-null   object 
 7   transaction_date  9835 non-null   object 
dtypes: float64(3), object(5)
memory usage: 702.7+ KB


In [27]:
# fill na with valid item value

df['item'].fillna(df['price_per_unit'].map(price_item), inplace = True)

<ipython-input-27-3e4abc14da58>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['item'].fillna(df['price_per_unit'].map(price_item), inplace = True)


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9994 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_id    9994 non-null   object 
 1   item              9994 non-null   object 
 2   quantity          9959 non-null   float64
 3   price_per_unit    9962 non-null   float64
 4   total_spent       9957 non-null   float64
 5   payment_method    7417 non-null   object 
 6   location          6732 non-null   object 
 7   transaction_date  9835 non-null   object 
dtypes: float64(3), object(5)
memory usage: 702.7+ KB


In [29]:
df['price_per_unit'].fillna(df['item'].map(item_price), inplace = True)

<ipython-input-29-1480f644bde3>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['price_per_unit'].fillna(df['item'].map(item_price), inplace = True)


Once we got the item and price per unit updated, we can further use these values to fill invalid values of other three columns that is price_per_unit, quantity, total_spent.

In [30]:
df[df['quantity'].isna()]

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,transaction_id,item,quantity,price_per_unit,total_spent,payment_method,location,transaction_date
236,TXN_8562645,Salad,NaN,5.0,NaN,NaN,In-store,2023-05-18
278,TXN_3229409,Juice,NaN,3.0,NaN,Cash,Takeaway,2023-04-15
629,TXN_9289174,Cake,NaN,3.0,12.0,Digital Wallet,In-store,2023-12-30
641,TXN_2962976,Juice,NaN,3.0,NaN,NaN,NaN,2023-03-17
738,TXN_8696094,Sandwich,NaN,4.0,NaN,NaN,Takeaway,2023-05-14
912,TXN_1575608,Sandwich,NaN,4.0,20.0,ERROR,Takeaway,2023-01-05
1008,TXN_7225428,Tea,NaN,1.5,3.0,Credit Card,Takeaway,2023-03-07
1436,TXN_7590801,Tea,NaN,1.5,6.0,Cash,Takeaway,ERROR
1482,TXN_3593060,Smoothie,NaN,4.0,16.0,Cash,NaN,2023-03-05
2330,TXN_3849488,Salad,NaN,5.0,5.0,NaN,In-store,2023-03-01


In [31]:
df['quantity'] = df.apply(lambda row: row['total_spent']/row['price_per_unit'] if pd.isna(row['quantity']) else row['quantity'], axis =1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9994 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_id    9994 non-null   object 
 1   item              9994 non-null   object 
 2   quantity          9974 non-null   float64
 3   price_per_unit    9994 non-null   float64
 4   total_spent       9957 non-null   float64
 5   payment_method    7417 non-null   object 
 6   location          6732 non-null   object 
 7   transaction_date  9835 non-null   object 
dtypes: float64(3), object(5)
memory usage: 702.7+ KB


In [32]:
df['price_per_unit'] = df.apply(lambda row: row['total_spent']/row['quantity'] if pd.isna(row['price_per_unit']) else row['price_per_unit'], axis =1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9994 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_id    9994 non-null   object 
 1   item              9994 non-null   object 
 2   quantity          9974 non-null   float64
 3   price_per_unit    9994 non-null   float64
 4   total_spent       9957 non-null   float64
 5   payment_method    7417 non-null   object 
 6   location          6732 non-null   object 
 7   transaction_date  9835 non-null   object 
dtypes: float64(3), object(5)
memory usage: 702.7+ KB


In [33]:
df['total_spent'] = df.apply(lambda row: row['price_per_unit']*row['quantity'] if pd.isna(row['total_spent']) else row['total_spent'], axis =1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9994 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_id    9994 non-null   object 
 1   item              9994 non-null   object 
 2   quantity          9974 non-null   float64
 3   price_per_unit    9994 non-null   float64
 4   total_spent       9974 non-null   float64
 5   payment_method    7417 non-null   object 
 6   location          6732 non-null   object 
 7   transaction_date  9835 non-null   object 
dtypes: float64(3), object(5)
memory usage: 702.7+ KB


We have cleaned up pretty good amount of data like quantity was 9959, now it's 9974, price per unit was 9962 now it's 9994, total_spent was 9957 now its 9974

In [34]:
df['payment_method'].unique()

array(['Credit Card', 'Cash', 'UNKNOWN', 'Digital Wallet', 'ERROR', nan],
      dtype=object)

We have cleaned up a significant amount of data:

- The `quantity` column was previously at 9959, and is now updated to 9974.
- The `price_per_unit` column was at 9962, and has now been updated to 9994.
- The `total_spent` column was at 9957, and is now updated to 9974.


In [35]:
strings_to_replace = ['UNKNOWN','ERROR', np.nan]
df.loc[df['payment_method'].isin(strings_to_replace),'payment_method' ]= 'Unknown'

In [36]:
df['payment_method'].unique()

array(['Credit Card', 'Cash', 'Unknown', 'Digital Wallet'], dtype=object)

In [37]:
df['location'].unique()

array(['Takeaway', 'In-store', 'UNKNOWN', nan, 'ERROR'], dtype=object)

We can do the same for in-store location as well.

In [38]:
df.loc[df['location'].isin(strings_to_replace),'location' ]= 'Unknown'

In [39]:
df['location'].unique()

array(['Takeaway', 'In-store', 'Unknown'], dtype=object)

For the `transaction_data`, I will drop any records that contain uninformative details. Since I am preparing this data for sales prediction, transactions without meaningful information are not useful for the analysis, especially when they lack transaction-specific data.


In [40]:
df.loc[df['transaction_date'].isin(strings_to_replace),'transaction_date' ]= np.nan

In [41]:
df['transaction_date'].unique()

array(['2023-09-08', '2023-05-16', '2023-07-19', '2023-04-27',
       '2023-06-11', '2023-03-31', '2023-10-06', '2023-10-28',
       '2023-07-28', '2023-12-31', '2023-11-07', nan, '2023-05-03',
       '2023-06-01', '2023-03-21', '2023-11-15', '2023-06-10',
       '2023-02-24', '2023-03-25', '2023-01-15', '2023-04-04',
       '2023-03-30', '2023-12-01', '2023-09-18', '2023-06-03',
       '2023-12-13', '2023-04-20', '2023-04-10', '2023-03-11',
       '2023-06-02', '2023-11-06', '2023-08-15', '2023-10-09',
       '2023-05-28', '2023-07-17', '2023-04-29', '2023-06-08',
       '2023-06-29', '2023-04-17', '2023-12-22', '2023-01-10',
       '2023-10-02', '2023-02-23', '2023-03-22', '2023-11-03',
       '2023-03-02', '2023-06-26', '2023-05-02', '2023-09-05',
       '2023-01-08', '2023-03-15', '2023-11-25', '2023-12-05',
       '2023-03-19', '2023-06-27', '2023-04-19', '2023-10-07',
       '2023-09-30', '2023-05-27', '2023-11-18', '2023-10-20',
       '2023-10-03', '2023-10-27', '2023-04-06', '

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9994 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_id    9994 non-null   object 
 1   item              9994 non-null   object 
 2   quantity          9974 non-null   float64
 3   price_per_unit    9994 non-null   float64
 4   total_spent       9974 non-null   float64
 5   payment_method    9994 non-null   object 
 6   location          9994 non-null   object 
 7   transaction_date  9534 non-null   object 
dtypes: float64(3), object(5)
memory usage: 702.7+ KB


In [43]:
df.dropna(subset=['transaction_date'], inplace =True)

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9534 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_id    9534 non-null   object 
 1   item              9534 non-null   object 
 2   quantity          9514 non-null   float64
 3   price_per_unit    9534 non-null   float64
 4   total_spent       9514 non-null   float64
 5   payment_method    9534 non-null   object 
 6   location          9534 non-null   object 
 7   transaction_date  9534 non-null   object 
dtypes: float64(3), object(5)
memory usage: 670.4+ KB


I believe we've made significant progress in cleaning the data. Most of our columns now have values that match the length of the column. However, the `quantity` and `total_spent` columns still contain **20** missing (NaN) values which is fine for prediction.
